In [1]:
from datasets import load_dataset

data_files = {
    "train": "indosum/train.04.jsonl",
    "validation": "indosum/dev.04.jsonl",
    "test": "indosum/test.04.jsonl",   
}

# Muat dataset
dataset = load_dataset("json", data_files=data_files)
print(dataset)

/home/rakhargo/miniconda3/envs/cudaTF/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['category', 'gold_labels', 'id', 'paragraphs', 'source', 'source_url', 'summary'],
        num_rows: 14272
    })
    validation: Dataset({
        features: ['category', 'gold_labels', 'id', 'paragraphs', 'source', 'source_url', 'summary'],
        num_rows: 750
    })
    test: Dataset({
        features: ['category', 'gold_labels', 'id', 'paragraphs', 'source', 'source_url', 'summary'],
        num_rows: 3752
    })
})


In [2]:
train_data = dataset['train']
print(train_data[0])

validation_data = dataset['validation']
print(validation_data[0])

test_data = dataset['test']
print(test_data[0])

{'category': 'tajuk utama', 'gold_labels': [[True], [False], [False], [True], [False], [True, True], [False, False, False], [False, False, False], [False], [False, False]], 'id': '1512964800-dirut-bri-ingin-holding-bumn-jasa-keuangan-segera-', 'paragraphs': [[['Setelah', 'selesai', 'membentuk', 'holding', 'BUMN', 'pertambangan', ',', 'kini', 'pemerintah', 'tengah', 'menggarap', 'holding', '-', 'holding', 'lainnya', ',', 'salah', 'satunya', 'holding', 'BUMN', 'jasa', 'keuangan', '.']], [['PT', 'Danareksa', '(', 'Persero', ')', 'akan', 'menjadi', 'induk', 'holding', 'membawahi', '4', 'Bank', 'BUMN', ',', 'yaitu', 'BNI', ',', 'BRI', ',', 'BTN', ',', 'dan', 'Bank', 'Mandiri', '.']], [['Selain', 'itu', ',', 'perusahaan', 'pelat', 'merah', 'lain', 'yang', 'bergerak', 'di', 'sektor', 'jasa', 'keuangan', 'seperti', 'PT', 'Jalin', 'Pembayaran', 'Nusantara', '(', 'JPN', ')', ',', 'PT', 'Permodalan', 'Nasional', 'Madani', '(', 'Persero', ')', '(', 'PMN', ')', ',', 'dan', 'PT', 'Pegadaian', '(', '

In [ ]:
def join_data(dataset):
    documents = []
    summaries = []
    for item in dataset:
        document = [" ".join(sent) for para in item["paragraphs"] for sent in para]
        summary = " ".join([" ".join(sent) for sent in item["summary"]])
        
        documents.append(document)
        summaries.append(summary)
    
    return documents, summaries

# Memproses data
train_docs, train_summaries = join_data(train_data)
val_docs, val_summaries = join_data(validation_data)
test_docs, test_summaries = join_data(test_data)

In [4]:
print(train_docs[0])
print(train_summaries[0])
len(train_docs)

['Setelah selesai membentuk holding BUMN pertambangan , kini pemerintah tengah menggarap holding - holding lainnya , salah satunya holding BUMN jasa keuangan .', 'PT Danareksa ( Persero ) akan menjadi induk holding membawahi 4 Bank BUMN , yaitu BNI , BRI , BTN , dan Bank Mandiri .', 'Selain itu , perusahaan pelat merah lain yang bergerak di sektor jasa keuangan seperti PT Jalin Pembayaran Nusantara ( JPN ) , PT Permodalan Nasional Madani ( Persero ) ( PMN ) , dan PT Pegadaian ( Persero ) juga akan bergabung di dalam holding BUMN jasa keuangan .', 'Direktur Utama PT Bank Rakyat Indonesia Tbk ( BBRI ) , Suprajarto , berharap pembentukan holding BUMN jasa keuangan segera terealisasi .', '" Kalau saya berharap lebih cepat lebih baik karena manfaatnya banyak , " kata Suprajarto di sela-sela acara BRI Run 2017 Bogor Series , Kebun Raya Bogor , Minggu ( 10 / 12 ) .', 'Orang nomor satu BRI ini mengatakan , jika holding jasa keuangan cepat terbentuk , pengadaan ATM bisa jauh lebih efisien secar

14272

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np

# Split data
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_docs, train_summaries, test_size=0.2, random_state=42
)

# Load Pretrained T5 Model and Tokenizer
model_name = "t5-small"  # Anda bisa mencoba t5-base atau t5-large
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)

# Tokenizing Dataset
def preprocess_data(texts, summaries, tokenizer, max_input_length=512, max_output_length=128):
    inputs = tokenizer(
        texts, max_length=max_input_length, padding="max_length", truncation=True, return_tensors="tf"
    )
    targets = tokenizer(
        summaries, max_length=max_output_length, padding="max_length", truncation=True, return_tensors="tf"
    )
    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": targets["input_ids"],
    }

train_data = preprocess_data(train_texts, train_labels, tokenizer)
val_data = preprocess_data(val_texts, val_labels, tokenizer)

ImportError: /home/rakhargo/miniconda3/envs/cudaTF/lib/python3.9/site-packages/torch/lib/../../nvidia/cusparse/lib/libcusparse.so.12: undefined symbol: __nvJitLinkComplete_12_4, version libnvJitLink.so.12

In [ ]:

# Create TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": train_data["input_ids"], "attention_mask": train_data["attention_mask"]},
    train_data["labels"]
)).batch(8)

val_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": val_data["input_ids"], "attention_mask": val_data["attention_mask"]},
    val_data["labels"]
)).batch(8)

# Compile Model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss_fn)

# Training
epochs = 3
history = model.fit(train_dataset, validation_data=val_dataset, epochs=epochs)

# Evaluation
for batch in val_dataset.take(1):
    input_ids = batch[0]["input_ids"]
    attention_mask = batch[0]["attention_mask"]
    generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=128)
    predictions = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    print("Generated Summaries:", predictions)